In [ ]:
PART 1

In [2]:
import numpy as np
import pandas as pd
import time
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy import stats  

In [3]:
TRAIN_PATH = "income.train.5k.csv"
DEV_PATH   = "income.dev.csv"
TEST_PATH  = "income.test.blind.csv"

train = pd.read_csv(TRAIN_PATH)
dev   = pd.read_csv(DEV_PATH)
test  = pd.read_csv(TEST_PATH)


In [4]:
train.columns

Index(['id', 'age', 'sector', 'edu', 'marriage', 'occupation', 'race', 'sex',
       'hours', 'country', 'target'],
      dtype='object')

In [5]:
dev.columns

Index(['id', 'age', 'sector', 'edu', 'marriage', 'occupation', 'race', 'sex',
       'hours', 'country', 'target'],
      dtype='object')

In [6]:
test.columns

Index(['id', 'age', 'sector', 'edu', 'marriage', 'occupation', 'race', 'sex',
       'hours', 'country'],
      dtype='object')

In [7]:
def positive_rate(df):
    a = (df['target'] == '>50K').mean() * 100
    return a

print("Train positive: %.2f%%" % positive_rate(train))
print("Dev   positive: %.2f%%" % positive_rate(dev))

Train positive: 25.02%
Dev   positive: 23.60%


In [8]:
youngest = int(train['age'].min())
oldest   = int(train['age'].max())

H = 'hours'
min_hrs  = int(train[H].min())
max_hrs  = int(train[H].max())

print("yougest age is: ", youngest)  # youngest age
print("oldest age is: ", oldest)    # oldest age
print("Least hours are: ", min_hrs)   # least hours
print("Max hours are: ",max_hrs)   # most hours


yougest age is:  17
oldest age is:  90
Least hours are:  1
Max hours are:  99


In [9]:
youngest = int(dev['age'].min())
oldest   = int(dev['age'].max())

H = 'hours'
min_hrs  = int(dev[H].min())
max_hrs  = int(dev[H].max())

print("yougest age is: ", youngest)  # youngest age
print("oldest age is: ", oldest)    # oldest age
print("Least hours are: ", min_hrs)   # least hours
print("Max hours are: ",max_hrs)   # most hours


yougest age is:  17
oldest age is:  90
Least hours are:  2
Max hours are:  99


In [10]:
Part 2

SyntaxError: invalid syntax (3944969287.py, line 1)

In [11]:
X_train = train.drop(columns=['id', 'target'])
y_train = (train['target'] == '>50K').astype(int)
X_dev = dev.drop(columns=['id', 'target'])
y_dev = (dev['target'] == '>50K').astype(int)
X_test = test.drop(columns=['id'])

In [12]:
all_feature_columns = ['age', 'hours', 'sector', 'edu', 'marriage', 'occupation', 'race', 'sex', 'country']

In [13]:
naive_preprocessor = ColumnTransformer( 
    transformers=[
        ('all_binner', OneHotEncoder(handle_unknown='ignore', sparse_output=False), all_feature_columns) ])

In [14]:
# for data pre-processing:
X_train_naive_encoded = naive_preprocessor.fit_transform(X_train)
X_dev_naive_encoded = naive_preprocessor.transform(X_dev)

In [15]:
for k in range(1, 101, 2):
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_naive_encoded, y_train)
    
    y_train_pred = clf.predict(X_train_naive_encoded)
    y_dev_pred = clf.predict(X_dev_naive_encoded)
    
    train_err = (1 - accuracy_score(y_train, y_train_pred)) * 100
    dev_err = (1 - accuracy_score(y_dev, y_dev_pred)) * 100
    

    train_pos_rate = np.mean(y_train_pred) * 100
    dev_pos_rate = np.mean(y_dev_pred) * 100
    
    print(f"k={k:<3} train_err = {train_err:4.1f}% (Positive +:{train_pos_rate:4.1f}%)   dev_err= {dev_err:4.1f}% (positive +:{dev_pos_rate:4.1f}%)")


k=1   train_err =  1.5% (Positive +:25.1%)   dev_err= 23.2% (positive +:24.8%)
k=3   train_err = 11.5% (Positive +:23.2%)   dev_err= 18.0% (positive +:22.2%)
k=5   train_err = 14.4% (Positive +:22.9%)   dev_err= 16.5% (positive +:19.7%)
k=7   train_err = 15.5% (Positive +:21.9%)   dev_err= 16.9% (positive +:20.9%)
k=9   train_err = 16.6% (Positive +:21.7%)   dev_err= 16.1% (positive +:19.9%)
k=11  train_err = 16.9% (Positive +:20.8%)   dev_err= 16.8% (positive +:18.8%)
k=13  train_err = 16.7% (Positive +:20.3%)   dev_err= 16.5% (positive +:18.1%)
k=15  train_err = 17.0% (Positive +:19.9%)   dev_err= 16.3% (positive +:18.5%)
k=17  train_err = 17.0% (Positive +:19.4%)   dev_err= 16.3% (positive +:18.7%)
k=19  train_err = 17.1% (Positive +:19.2%)   dev_err= 16.1% (positive +:18.5%)
k=21  train_err = 17.2% (Positive +:19.1%)   dev_err= 16.2% (positive +:18.8%)
k=23  train_err = 17.5% (Positive +:19.1%)   dev_err= 16.0% (positive +:18.2%)
k=25  train_err = 17.4% (Positive +:18.8%)   dev_err

In [16]:
X_train_naive_encoded

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [17]:
feature_dimension = X_train_naive_encoded.shape[1]

In [18]:
print(f"\nThe feature dimensions after naive-binarization is: {feature_dimension}")


The feature dimensions after naive-binarization is: 230


In [19]:
#conversion into csv file

In [20]:
best_k = 69 

In [21]:
full_train_df = pd.concat([train, dev], ignore_index=True)
X_train_full_csv = full_train_df.drop(columns=['id', 'target'])
y_train_full_csv = (full_train_df['target'] == '>50K').astype(int)

In [22]:
#Fitting naive bayes preprocessor on combined  dataset:
X_best_train_naive_encoded = naive_preprocessor.fit_transform(X_train_full_csv)
X_test_encoded = naive_preprocessor.transform(X_test)

In [23]:
print(f"Training final model with k={best_k}")
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_best_train_naive_encoded, y_train_full_csv)

Training final model with k=69


,n_neighbors,69
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [24]:
final_model = KNeighborsClassifier(n_neighbors=best_k)

In [25]:
final_model.fit(X_best_train_naive_encoded, y_train_full_csv)
print("Model training complete.")

Model training complete.


In [26]:
test_predictions_int = final_model.predict(X_test_encoded)
print("prediction for testing complete.")

prediction for testing complete.


In [27]:
test_predictions_labels = ['>50K' if pred == 1 else '<=50K' for pred in test_predictions_int]

In [28]:
submission_df = test.copy()
submission_df['target'] = test_predictions_labels

submission_df = submission_df[train.columns]
 
output_filename = 'income.test.predicted.csv'
submission_df.to_csv(output_filename, index=False)

In [ ]:
PART3: 

In [29]:
'''
Already defined above: 
X_train = train.drop(columns=['id', 'target'])
y_train = (train['target'] == '>50K').astype(int)
X_dev = dev.drop(columns=['id', 'target'])
y_dev = (dev['target'] == '>50K').astype(int)
'''

"\nAlready defined above: \nX_train = train.drop(columns=['id', 'target'])\ny_train = (train['target'] == '>50K').astype(int)\nX_dev = dev.drop(columns=['id', 'target'])\ny_dev = (dev['target'] == '>50K').astype(int)\n"

In [30]:
numeric_feat = ['age', 'hours']
categorical_feat = ['sector', 'edu', 'marriage', 'occupation', 'race', 'sex', 'country']

In [31]:
smart_preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_feat), # this is seperate for keeping the numerical fields as they are
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_feat) 
    ])

In [32]:
#now doing the prediction using the smart binerization and storing it in a new variable
smart_preprocessor.fit(X_train)
X_train_smarter = smart_preprocessor.transform(X_train)
X_dev_smarter = smart_preprocessor.transform(X_dev)

In [33]:
feature_dimension = X_train_smarter.shape[1]
print(f"\nThe feature dimension after binarization is: {feature_dimension}")


The feature dimension after binarization is: 92


In [34]:
for k in range(1, 101, 2):  
    
    # Initialize the KNeighborsClassifier with the current value of k
    clf = KNeighborsClassifier(n_neighbors=k)
    
    clf.fit(X_train_smarter, y_train)
    
    y_train_pred = clf.predict(X_train_smarter)
    y_dev_pred = clf.predict(X_dev_smarter)
    
    train_err = (1 - accuracy_score(y_train, y_train_pred)) * 100
    dev_err = (1 - accuracy_score(y_dev, y_dev_pred)) * 100
    train_pos_rate = np.mean(y_train_pred) * 100
    dev_pos_rate = np.mean(y_dev_pred) * 100
    
    print(f"k={k:<3} train_err = {train_err:4.1f}% (Positive +:{train_pos_rate:4.1f}%)   dev_err= {dev_err:4.1f}% (positive +:{dev_pos_rate:4.1f}%)")

k=1   train_err =  1.5% (Positive +:25.1%)   dev_err= 26.9% (positive +:27.3%)
k=3   train_err = 13.0% (Positive +:24.9%)   dev_err= 24.0% (positive +:25.0%)
k=5   train_err = 15.6% (Positive +:23.2%)   dev_err= 23.4% (positive +:25.6%)
k=7   train_err = 16.8% (Positive +:22.8%)   dev_err= 23.3% (positive +:23.9%)
k=9   train_err = 18.3% (Positive +:22.1%)   dev_err= 22.2% (positive +:22.2%)
k=11  train_err = 18.8% (Positive +:21.9%)   dev_err= 21.9% (positive +:22.1%)
k=13  train_err = 19.0% (Positive +:21.9%)   dev_err= 21.7% (positive +:20.9%)
k=15  train_err = 19.2% (Positive +:21.4%)   dev_err= 21.5% (positive +:20.9%)
k=17  train_err = 19.5% (Positive +:20.9%)   dev_err= 22.1% (positive +:20.1%)
k=19  train_err = 19.7% (Positive +:20.8%)   dev_err= 21.4% (positive +:19.6%)
k=21  train_err = 19.8% (Positive +:20.4%)   dev_err= 21.7% (positive +:19.7%)
k=23  train_err = 20.3% (Positive +:19.8%)   dev_err= 22.3% (positive +:19.7%)
k=25  train_err = 20.1% (Positive +:19.4%)   dev_err

In [35]:
X_train_smarter

array([[50., 13.,  0., ...,  1.,  0.,  0.],
       [38., 40.,  0., ...,  1.,  0.,  0.],
       [53., 40.,  0., ...,  1.,  0.,  0.],
       ...,
       [61.,  5.,  0., ...,  1.,  0.,  0.],
       [42., 40.,  0., ...,  1.,  0.,  0.],
       [21., 40.,  0., ...,  1.,  0.,  0.]])

In [73]:
X_dev_smarter

array([[45., 45.,  1., ...,  1.,  0.,  0.],
       [33., 20.,  0., ...,  0.,  0.,  0.],
       [19., 40.,  0., ...,  1.,  0.,  0.],
       ...,
       [20., 40.,  0., ...,  1.,  0.,  0.],
       [22., 35.,  0., ...,  1.,  0.,  0.],
       [58., 40.,  0., ...,  1.,  0.,  0.]])

In [37]:
# now implementing the minmax scaler for numeric_feat
scaled_preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(feature_range=(0, 2)), numeric_feat), #MInMaxscaler
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_feat)
    ])

In [38]:
#Preparing smartly binarized AND scaled data.
scaled_preprocessor.fit(X_train)
X_train_scale = scaled_preprocessor.transform(X_train)
X_dev_scale = scaled_preprocessor.transform(X_dev)
X_test_scale = scaled_preprocessor.transform(X_test)

In [39]:
#checking the feature dimention for smart binerization and scaled data
print(f"The feature dimension is: {X_train_scale.shape[1]}")

The feature dimension is: 92


In [40]:
#printing the values of k for scaled_preprocessor
for k in range(1, 101, 2):
    
    clf = KNeighborsClassifier(n_neighbors=k)
   
    clf.fit(X_train_scale, y_train)

    y_train_pred = clf.predict(X_train_scale)
    y_dev_pred = clf.predict(X_dev_scale)
    
    train_err = (1 - accuracy_score(y_train, y_train_pred)) * 100
    dev_err = (1 - accuracy_score(y_dev, y_dev_pred)) * 100
    train_pos_rate = np.mean(y_train_pred) * 100
    dev_pos_rate = np.mean(y_dev_pred) * 100
    
    # Print the results in the required format
    print(f"k={k:<3} train_err = {train_err:4.1f}% (Positive + = {train_pos_rate:4.1f}%)   dev_err = {dev_err:4.1f}% (Positive+ = {dev_pos_rate:4.1f}%)")

k=1   train_err =  1.5% (Positive + = 25.1%)   dev_err = 23.8% (Positive+ = 27.0%)
k=3   train_err = 11.5% (Positive + = 24.2%)   dev_err = 19.2% (Positive+ = 26.0%)
k=5   train_err = 13.8% (Positive + = 23.9%)   dev_err = 18.4% (Positive+ = 24.6%)
k=7   train_err = 14.1% (Positive + = 23.8%)   dev_err = 16.9% (Positive+ = 24.1%)
k=9   train_err = 15.5% (Positive + = 23.6%)   dev_err = 15.9% (Positive+ = 22.3%)
k=11  train_err = 16.3% (Positive + = 23.5%)   dev_err = 16.3% (Positive+ = 21.1%)
k=13  train_err = 16.4% (Positive + = 23.5%)   dev_err = 16.4% (Positive+ = 21.8%)
k=15  train_err = 16.4% (Positive + = 23.1%)   dev_err = 15.7% (Positive+ = 21.9%)
k=17  train_err = 16.6% (Positive + = 22.8%)   dev_err = 15.9% (Positive+ = 21.7%)
k=19  train_err = 16.9% (Positive + = 22.4%)   dev_err = 16.3% (Positive+ = 20.9%)
k=21  train_err = 17.0% (Positive + = 22.3%)   dev_err = 15.9% (Positive+ = 21.1%)
k=23  train_err = 17.2% (Positive + = 22.3%)   dev_err = 15.4% (Positive+ = 21.6%)
k=25

In [41]:
X_train_scale

array([[0.90410959, 0.24489796, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.57534247, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.98630137, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [1.20547945, 0.08163265, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.68493151, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.10958904, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ]])

In [42]:
X_dev_scale

array([[0.76712329, 0.89795918, 1.        , ..., 1.        , 0.        ,
        0.        ],
       [0.43835616, 0.3877551 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05479452, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.08219178, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.1369863 , 0.69387755, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.12328767, 0.79591837, 0.        , ..., 1.        , 0.        ,
        0.        ]])

In [43]:
# CSV file conversion for best model

In [44]:
best_k = 41  # This k gave the lowest dev error of 14.5%.
full_train_df = pd.concat([train, dev], ignore_index=True)
X_train_full_csv = full_train_df.drop(columns=['id', 'target'])
y_train_full_csv = (full_train_df['target'] == '>50K').astype(int)


In [45]:
X_full_train_scaled = scaled_preprocessor.fit_transform(X_train_full_csv)
X_test = test.drop(columns=['id'])

In [46]:
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_full_train_scaled, y_train_full_csv)

,n_neighbors,41
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [47]:
X_test_scale = scaled_preprocessor.transform(X_test)

In [48]:
test_predictions_int_new = final_model.predict(X_test_scale)

In [49]:
test_predictions_labels_new = ['>50K' if pred == 1 else '<=50K' for pred in test_predictions_int_new]

In [50]:
submission_df = test.copy()
submission_df['target'] = test_predictions_labels

In [51]:
submission_df = test.copy()
submission_df['target'] = test_predictions_labels
submission_df = submission_df[train.columns]
output_filename = 'income.test.predicted.csv'
submission_df.to_csv(output_filename, index=False)

In [ ]:
PART 4:

In [52]:
query_person = X_dev_scale[0]

In [53]:
#took it from the hw_file 
neighbor_1 = X_train_scale[4872]
neighbor_2 = X_train_scale[4787]
neighbor_3 = X_train_scale[2591]

In [54]:
#now using sklearn 
# finding the euclidean distance
knn_euclidean = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_euclidean.fit(X_train_scale, y_train)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


In [55]:
distances_e, indices_e = knn_euclidean.kneighbors(query_person.reshape(1, -1))

In [56]:
print("Euclidean Distances of all the neighbors:", distances_e[0])
print("Indices all the neighbors:", indices_e[0])
# Matching the values as per the expectations

Euclidean Distances of all the neighbors: [0.33441929 1.41527469 1.41674697]
Indices all the neighbors: [4872 4787 2591]


In [57]:
#now using sklearn 
# finding the Manhattan distance
knn_man = KNeighborsClassifier(n_neighbors=3, metric='manhattan')
knn_man.fit(X_train_scale, y_train)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'manhattan'
,metric_params,None
,n_jobs,None


In [58]:
distances_m, indices_m = knn_man.kneighbors(query_person.reshape(1, -1))

In [59]:
print("Manhattan Distances of all the neighbors:", distances_m[0])
print("Indices of Neighbors of all the neighbors:", indices_m[0])
# Matching the values as per the expectations

Manhattan Distances of all the neighbors: [0.38999161 2.05479452 2.10204082]
Indices of Neighbors of all the neighbors: [4872 4787 1084]


In [60]:
#calculating knn_distance from scratch (used the hint)
dist_e_1 = np.linalg.norm(query_person - neighbor_1)
dist_e_2 = np.linalg.norm(query_person - neighbor_2)
dist_e_3 = np.linalg.norm(query_person - neighbor_3)

In [61]:
print("The Euclidean Distances are:",[dist_e_1, dist_e_2, dist_e_3])

The Euclidean Distances are: [0.3344192869821078, 1.4152746869361013, 1.4167469717499104]


In [62]:
#to calculate manhanttan distance 
# manhattan = sum (absolute(coordinate1 - coordinate2))
dist_m_1 = np.sum(np.abs(query_person - neighbor_1))
dist_m_2 = np.sum(np.abs(query_person - neighbor_2))
dist_m_3 = np.sum(np.abs(query_person - neighbor_3))

In [63]:
print("The Manhattan Distances are", [dist_m_1, dist_m_2, dist_m_3])

The Manhattan Distances are [0.38999161308358965, 2.054794520547945, 2.102599944087224]


In [64]:
#Building a Knn Model of my own

In [65]:
#Used the X_train_scale,X_dev_scale,X_test_scale from previous parts
y_train_num = y_train.to_numpy()
y_dev_num = y_dev.to_numpy()

In [66]:
class KNNClassifierSuper:
  
    def __init__(self, n_neighbors=5):
        self.k_e = n_neighbors

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for test_point in X_test:
            distances = np.linalg.norm(self.X_train - test_point, axis=1)
            
            topk_indices = np.argpartition(distances, self.k_e)[:self.k_e]
            
            topk_labels = self.y_train[topk_indices]
    
            prediction = stats.mode(topk_labels)[0]
            predictions.append(prediction)
        return np.array(predictions)


In [67]:
print("Evaluating from-scratch k-NN with Euclidean Distance from k=1 to k= 99")
euclidean_results = {}

#start time
total_start_time = time.time()
for k in range(1, 101, 2):
    my_clf = KNNClassifierSuper(n_neighbors=k)

    
    my_clf.fit(X_train_scale, y_train_num)
    
    y_train_pred_euc = my_clf.predict(X_train_scale)
    y_dev_pred_euc = my_clf.predict(X_dev_scale)



    train_err_euc = (1 - accuracy_score(y_train_num, y_train_pred_euc)) * 100
    dev_err_euc = (1 - accuracy_score(y_dev_num, y_dev_pred_euc)) * 100
    train_pos_rate = np.mean(y_train_pred_euc) * 100
    dev_pos_rate = np.mean(y_dev_pred_euc) * 100
    

    euclidean_results[k] = dev_err_euc
    print(f"k={k:<3} train_err {train_err_euc:4.1f}% (+:{train_pos_rate:4.1f}%)   dev_err {dev_err_euc:4.1f}% (+:{dev_pos_rate:4.1f}%)")

# total time taken by model 
total_end_time = time.time()
time_taken = total_end_time - total_start_time
print(f"\nTotal time taken for the entire loop = {time_taken:.2f} seconds")

Evaluating from-scratch k-NN with Euclidean Distance from k=1 to k= 99
k=1   train_err  1.5% (+:25.1%)   dev_err 23.9% (+:27.3%)
k=3   train_err 11.4% (+:23.9%)   dev_err 19.6% (+:25.4%)
k=5   train_err 13.7% (+:23.6%)   dev_err 17.8% (+:24.4%)
k=7   train_err 14.3% (+:23.8%)   dev_err 16.6% (+:24.0%)
k=9   train_err 15.4% (+:23.6%)   dev_err 15.3% (+:22.5%)
k=11  train_err 16.5% (+:23.6%)   dev_err 16.4% (+:21.2%)
k=13  train_err 16.5% (+:23.5%)   dev_err 16.7% (+:21.9%)
k=15  train_err 16.4% (+:23.1%)   dev_err 15.9% (+:21.9%)
k=17  train_err 16.7% (+:22.8%)   dev_err 15.8% (+:21.6%)
k=19  train_err 16.7% (+:22.5%)   dev_err 16.4% (+:20.8%)
k=21  train_err 16.9% (+:22.2%)   dev_err 16.2% (+:21.0%)
k=23  train_err 17.0% (+:22.3%)   dev_err 15.5% (+:21.7%)
k=25  train_err 16.9% (+:22.4%)   dev_err 15.6% (+:21.4%)
k=27  train_err 16.9% (+:22.2%)   dev_err 15.6% (+:20.8%)
k=29  train_err 17.0% (+:21.8%)   dev_err 15.1% (+:21.1%)
k=31  train_err 16.9% (+:21.9%)   dev_err 15.2% (+:21.2%)
k

In [68]:
# model training using Manhattan distance

In [69]:
class KNNClassifierSuper2:
  
    def __init__(self, n_neighbors=5):
        self.k = n_neighbors

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for test_point in X_test:
   
            distances = np.sum(np.abs(self.X_train - test_point), axis=1)
        
            topk_indices = np.argpartition(distances, self.k)[:self.k]
            topk_labels = self.y_train[topk_indices]
    
            prediction = stats.mode(topk_labels)[0]
            predictions.append(prediction)
        return np.array(predictions)

In [70]:
print("Evaluating from-scratch k-NN with Manhattan Distance for k=1 to 99")
manhattan_results = {}

total_start_time = time.time() 

for k in range(1, 101, 2):
    
    my_clf2 = KNNClassifierSuper2(n_neighbors=k)
    my_clf2.fit(X_train_scale, y_train_num)
    
    y_train_pred_man = my_clf2.predict(X_train_scale)
    y_dev_pred_man = my_clf2.predict(X_dev_scale)
    
    train_err_man = (1 - accuracy_score(y_train_num, y_train_pred_man)) * 100
    dev_err_man = (1 - accuracy_score(y_dev_num, y_dev_pred_man)) * 100
    train_pos_rate = np.mean(y_train_pred_man) * 100
    dev_pos_rate = np.mean(y_dev_pred_man) * 100
    
    manhattan_results[k] = dev_err_man
    
    print(f"k={k:<3} train_err {train_err_man:4.1f}% (+:{train_pos_rate:4.1f}%)   dev_err {dev_err_man:4.1f}% (+:{dev_pos_rate:4.1f}%)")

total_end_time = time.time()
time_taken = total_end_time - total_start_time

# to print the time taken to train the model
print(f"\nTime taken to predict on train and dev sets = {time_taken:.2f} seconds")

Evaluating from-scratch k-NN with Manhattan Distance for k=1 to 99
k=1   train_err  1.5% (+:25.1%)   dev_err 24.0% (+:27.2%)
k=3   train_err 11.6% (+:23.9%)   dev_err 19.7% (+:26.1%)
k=5   train_err 14.0% (+:23.9%)   dev_err 17.6% (+:25.0%)
k=7   train_err 14.5% (+:24.0%)   dev_err 16.6% (+:24.0%)
k=9   train_err 15.5% (+:23.6%)   dev_err 16.2% (+:22.2%)
k=11  train_err 16.2% (+:23.4%)   dev_err 16.3% (+:21.9%)
k=13  train_err 16.5% (+:23.6%)   dev_err 16.5% (+:22.3%)
k=15  train_err 16.8% (+:22.8%)   dev_err 16.3% (+:21.7%)
k=17  train_err 16.9% (+:22.8%)   dev_err 15.5% (+:21.1%)
k=19  train_err 17.0% (+:22.5%)   dev_err 16.0% (+:21.0%)
k=21  train_err 17.0% (+:22.2%)   dev_err 16.6% (+:21.2%)
k=23  train_err 17.1% (+:22.3%)   dev_err 16.2% (+:21.6%)
k=25  train_err 17.0% (+:22.1%)   dev_err 15.7% (+:21.1%)
k=27  train_err 16.9% (+:21.8%)   dev_err 15.8% (+:20.6%)
k=29  train_err 16.8% (+:21.3%)   dev_err 16.0% (+:20.6%)
k=31  train_err 17.0% (+:21.2%)   dev_err 15.9% (+:20.5%)
k=33 

In [71]:
# to calculate the final test positive ratio 

# Defining the best model's k...
best_k = 41
best_metric = 'manhattan'

# Combine train and dev sets for final training
full_train_df = pd.concat([train, dev], ignore_index=True)
X_full_train = full_train_df.drop(columns=['id', 'target'])
y_full_train = (full_train_df['target'] == '>50K').astype(int).to_numpy()

# Using the smart+scaled preprocessor and fit/transform the data

X_full_train_scaled = scaled_preprocessor.fit_transform(X_full_train)
X_test_scaled = scaled_preprocessor.transform(test.drop(columns=['id']))

# Train the final from-scratch model (using Manhattan)

final_model = KNNClassifierSuper2(n_neighbors=best_k)
final_model.fit(X_full_train_scaled, y_full_train)

# Make predictions on the blind test set
test_predictions = final_model.predict(X_test_scaled)

# Calculate the positive ratio
test_positive_ratio = np.mean(test_predictions) * 100

print(f"The positive ratio on the test set is: {test_positive_ratio:.2f}%")

The positive ratio on the test set is: 20.30%


In [72]:
# 7 conversion into a csv file
test_predictions_labels = ['>50K' if pred == 1 else '<=50K' for pred in test_predictions]

submission_df = test.copy()
submission_df['target'] = test_predictions_labels
submission_df = submission_df[train.columns]

output_file = 'income.test.predicted.csv'
submission_df.to_csv(output_file, index=False)